In [1]:
import pandas as pd
import numpy as np
import pandas.io.data as web
import pickle
%matplotlib inline

C:\Users\alifa\Anaconda2\lib\site-packages\pandas\io\data.py:35: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)


In [15]:
def PMA_Strategy_daily(symbol,short_PMA=14,med_PMA=30,long_pma=50):
    try:
        
        priceDF=pd.DataFrame()
        priceDF=web.DataReader(symbol,data_source='yahoo',start='2010-02-28')
        priceDF.dropna(inplace=True)
        
        priceDF['HCL_Ave']=list(map(lambda x,y,z:(x+y+z)/3,priceDF['Adj Close'],priceDF['High'],priceDF['Low']))     
        #this is the way that we apply a formula on a series
        
        priceDF['HCL_MA14']=priceDF['HCL_Ave'].ewm(span=short_PMA).mean()
        priceDF['HCL_MA30']=priceDF['HCL_Ave'].ewm(span=med_PMA).mean()
        priceDF['HCL_MA50']=priceDF['HCL_Ave'].ewm(span=long_pma).mean()
        
        if  priceDF['HCL_MA14'].iloc[-1]> priceDF['HCL_MA30'].iloc[-1]>priceDF['HCL_MA50'].iloc[-1]:
            return 'Yes'
        else:
            return 'No'

    except Exception as e:
        print ('PMA_Strategy_daily',str(e)) 

In [17]:
PMA_Strategy_daily('AAPL')

'Yes'

In [27]:
def PMA_Strategy_Intraday(symbol,short_PMA=14,med_PMA=30,long_pma=50):
    try:
        
        urlToVisit='http://www.google.com/finance/getprices?i=1800&p=10d&f=d,o,h,l,c,v&df=cpct&q='+symbol
        cols = ['openp', 'highp', 'lowp', 'closep','volume']
        priceDF = pd.read_csv(urlToVisit, index_col=0, parse_dates=True, dayfirst=True, header=None,
                         skiprows=8, names=cols)
        
        priceDF.dropna(inplace=True)
        priceDF['HCL_Ave']=list(map(lambda x,y,z:(x+y+z)/3,priceDF['closep'],priceDF['highp'],priceDF['lowp']))     
        
        priceDF['HCL_MA14']=priceDF['HCL_Ave'].ewm(span=short_PMA).mean()
        priceDF['HCL_MA30']=priceDF['HCL_Ave'].ewm(span=med_PMA).mean()
        priceDF['HCL_MA50']=priceDF['HCL_Ave'].ewm(span=long_pma).mean()
       
        if  (priceDF['HCL_MA14'].iloc[-1] > priceDF['HCL_MA30'].iloc[-1]>priceDF['HCL_MA50'].iloc[-1]) and \
        priceDF['HCL_MA14'].iloc[-1] > priceDF['HCL_MA14'].iloc[-2] and \
        priceDF['HCL_MA30'].iloc[-1] > priceDF['HCL_MA30'].iloc[-2] and \
        priceDF['HCL_MA50'].iloc[-1] > priceDF['HCL_MA50'].iloc[-2]:
            return 'Yes'
        else:
            return 'No'

    except Exception as e:
        print ('PMA_Strategy_Intraday',str(e))

In [43]:
PMA_Strategy_Intraday('AMG')

'No'

In [50]:
sp500List=pd.read_csv('http://data.okfn.org/data/core/s-and-p-500-companies/r/constituents.csv')
sp500List.rename(columns={'Symbol':'symbol'},inplace=True)
sp500List['HCL_Strategy']=sp500List.head(10)['symbol'].map(PMA_Strategy_Intraday)
sp500List=sp500List[sp500List['HCL_Strategy']=='Yes']
sp500List.head(10)

,symbol,Name,Sector,HCL_Strategy
3,ACN,Accenture plc,Information Technology,Yes
4,ATVI,Activision Blizzard,Information Technology,Yes
5,AYI,Acuity Brands Inc,Industrials,Yes
6,ADBE,Adobe Systems Inc,Information Technology,Yes
7,AAP,Advance Auto Parts,Consumer Discretionary,Yes
9,AET,Aetna Inc,Health Care,Yes


In [37]:
symbolExtration =pd.read_pickle('NYSE_Buy_Overweight.pickle')
symbolExtration.head()

,company,industry,sector,symbol,ANR,PE,FPE,ATP,PEG
0,3M Company,Medical/Dental Instruments,Health Care,MMM,Overweight,22.60,20.05,186.44,2.11
1,A.H. Belo Corporation,Newspapers/Magazines,Consumer Services,AHC,Buy,-9.48,22.68,N/A,NA
2,A10 Networks,Technology,n/a,ATEN,Overweight,-20.29,92.22,9.63,NA
3,AAR Corp.,Aerospace,Capital Goods,AIR,Overweight,23.67,19.51,36.00,NA
4,Aaron&#39;s,Technology,n/a,AAN,Overweight,15.54,12.07,35.43,NA


In [57]:
def stock_screening(market='sp500'):
    if market=='amex':
        symbolExtration =pd.read_pickle('NYSE_Buy_Overweight.pickle')
        symbolExtration.reset_index(drop=True,inplace=True)
    else:
        symbolExtration=pd.read_csv('http://data.okfn.org/data/core/s-and-p-500-companies/r/constituents.csv')
        symbolExtration.rename(columns={'Symbol':'symbol'},inplace=True)
        
    screenedSymbols=symbolExtration
    
      
    screenedSymbols['HCL_Strategy']=screenedSymbols['symbol'].map(PMA_Strategy_Intraday)
    screenedSymbols=screenedSymbols[screenedSymbols['HCL_Strategy']=='Yes']
    screenedSymbols.reset_index(drop=True,inplace=True)
    return screenedSymbols

In [59]:
stock_screening=stock_screening()

('PMA_Strategy_Intraday', 'single positional indexer is out-of-bounds')
('PMA_Strategy_Intraday', 'single positional indexer is out-of-bounds')


In [65]:
stock_screening.head()

,symbol,Name,Sector,HCL_Strategy
0,ACN,Accenture plc,Information Technology,Yes
1,ATVI,Activision Blizzard,Information Technology,Yes
2,AYI,Acuity Brands Inc,Industrials,Yes
3,ADBE,Adobe Systems Inc,Information Technology,Yes
4,AAP,Advance Auto Parts,Consumer Discretionary,Yes


In [69]:
stock_screening['Sector'].value_counts()

Consumer Discretionary         34
Information Technology         32
Energy                         24
Industrials                    22
Financials                     19
Health Care                    14
Utilities                      14
Real Estate                    12
Consumer Staples               12
Materials                       4
Telecommunications Services     1
Name: Sector, dtype: int64

In [72]:
import time
import urllib2
from urllib2 import urlopen
import re

In [76]:
def YahooFinanceFPE(stock):
    try:
        sourceCode=urllib2.urlopen('https://finance.yahoo.com/quote/'+stock+'/key-statistics?p='+stock).read()  
        siteBreak=sourceCode.split('forwardPE')[1].split('"')[6]
        return siteBreak

    except Exception, e:
        print 'failed in the YahooFinanceFPE', str(e)

In [79]:
def marketwatchAR(stock):
    try:
        sourceCode=urllib2.urlopen('http://www.marketwatch.com/investing/Stock/'+\
                                   stock+'/analystestimates?CountryCode=US').read()   #always put http:// at the beginning   
        siteBreak=sourceCode.split('Average Recommendation')[1].split('</td>')[1].split('>')[1].strip()  
        return siteBreak

    except Exception, e:
        print 'failed in the marketwatchAR', str(e)

In [82]:
def marketwatchTP(stock):
    try:
        sourceCode=urllib2.urlopen('http://www.marketwatch.com/investing/Stock/'+\
                                   stock+'/analystestimates?CountryCode=US').read()   #always put http:// at the beginning   
        siteBreak=sourceCode.split('Average Target Price')[1].split('</td>')[1]
        numberStart=siteBreak.find('>')   
        return siteBreak[numberStart+1:]

    except Exception, e:
        print 'failed in the marketwatchTP', str(e)

In [84]:
def PEGZack(stock):
    try:
        sourceCode=urllib2.urlopen('https://www.zacks.com/stock/quote/'+stock).read()    
        siteBreak=sourceCode.split('peg-ratio-ttm')[2].split('>')[1].split('<')[0]
        return siteBreak

    except Exception, e:
        print 'failed in the PEGZack', str(e)

In [85]:
PEGZack('GM')

'0.57'

In [78]:
YahooFinanceFPE('AAPL')

'14.73'

In [87]:
!conda install yahoo_finance

Fetching package metadata ...........




PackageNotFoundError: Package missing in current win-64 channels: 
  - yahoo_finance




In [86]:
from yahoo_finance import Share

ImportError: No module named yahoo_finance

In [130]:
def export_to_files(new_stocks):
    old_HCL=pd.read_pickle('Old_HCL.pickle')
    newAdded=new_stocks[~new_stocks['symbol'].isin(old_HCL['symbol'])].dropna()  #find the new stocks that are added
    newAdded.reset_index(drop=True,inplace=True)
    newAdded.dropna(inplace=True)
    old_HCL=open('Old_HCL.pickle','wb')
    pickle.dump(new_stocks,old_HCL)
    old_HCL.close()
    New_HCL=open('New_HCL.pickle','wb')
    pickle.dump(newAdded,New_HCL)
    New_HCL.close()
    return 

In [ ]:
def export_to_Excel():
        def divideP(x,y):
        try:
            return float(x.replace(',',''))/float(y.replace(',',''))
        except Exception as e:
            return 'NaN'
    
    try:
        newSymbolsAdded =pd.read_pickle('New_HCL.pickle')
        if newSymbolsAdded.empty:
            return newSymbolsAdded
        else:
            newSymbolsAdded['CMP']=newSymbolsAdded['symbol'].map(fe.YahooFinanceMP)
            newSymbolsAdded['daily']=newSymbolsAdded['symbol'].map(PMA_Strategy_daily)
            newSymbolsAdded['CMP/ATP']=list(map(divideP,newSymbolsAdded['CMP'],newSymbolsAdded['ATP']))
            del newSymbolsAdded['sector']
            newSymbolsAdded=newSymbolsAdded[['symbol','company','industry','ANR','PE','HCL_Strategy','FPE','PEG','ATP','CMP','CMP/ATP','daily']]
            #newSymbolsAdded.sort(['ANR','CMP/ATP','FPE','PEG'], ascending=[True, True,True,True], inplace=True)
            newSymbolsAdded.sort_values(by=['ANR','CMP/ATP','FPE','PEG'], ascending=[True, True,True,True], inplace=True)
            newSymbolsAdded.reset_index(drop=True,inplace=True)
            
            writer = pd.ExcelWriter('C:/Users/alifa/OneDrive/Trading_Strategies/NewPositiveMomentum.xlsx')
            newSymbolsAdded.to_excel(writer,sheet_name='New')
            writer.save()
            writer.close()
            
            return newSymbolsAdded

    except Exception as e:
        print ('importToExcel Function',str(e))
    